### Notebook For DS4400 Lecture 7

By: John Henry Rudden

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression as SKLinearRegression
import numpy as np
from numpy import ndarray

In [2]:
X = np.genfromtxt('./data/stock_prediction_data.csv', delimiter=',')
y = np.genfromtxt('./data/stock_price.csv', delimiter=',')
y = y.reshape(-1, 1)

### Splitting Data into Train, Validation, and Test Sets

Use train to train the model, validation to compare models, and test to evaluate the final selected model.

Note: using `random_state` to ensure reproducibility of splits.

In [3]:
X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_rest, y_rest, test_size=0.5, random_state=42)

### Scaling our Data

We will use SKLearn's StandardScaler to scale our data. This will make our data have a mean of 0 and a standard deviation of 1. 
It is important to scale our validation and test sets using the scaler that was fit to the training data. Since scaling is part of our preprocessing, we need to be consistent with the scaling across all of our data (train, validation, test, and future).

In [4]:
# use standard scaler to normalize the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

### Implementing a Linear Regression Model

In our data science class, we've learned that the objective for linear regression is formulated as:

$$\min_{w} L(w) =  \frac{1}{n} \sum_{i=1}^{n} (\phi(x_i)^Tw - y_i)^2$$

where $\phi(x_i)$ represents the feature vector of the $i^{th}$ observation, $w$ is the weight vector, and $y_i$ is the target value for the $i^{th}$ observation. We also covered that the derivative of the loss objective function \(L(w)\) with respect to the weight vector \(w\) is:

$$\frac{\partial L(w)}{\partial w} =  \frac{2}{n}\sum_{i=1}^{n} \phi(x_i)(\phi(x_i)^Tw - y_i)$$

However, for efficiency and readability, this notebook adopts a vectorized implementation for the derivative of the loss objective function. The vectorized derivative of the loss objective function with respect to \(w\) is given by:

$$\frac{\partial L(w)}{\partial w} = \frac{2}{n} \Phi^T(\Phi w - y)$$

where $\Phi$ denotes the feature matrix (with each row as a feature vector corresponding to an observation) and \(y\) is the vector of target values.

$$\Phi = \begin{bmatrix} \phi(x_1)^T \\ \phi(x_2)^T \\ \vdots \\ \phi(x_n)^T \end{bmatrix}, \quad y = \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}$$

Below is a derivation of the vectorized implementation of the objective function derivative. You can skip the derivation if you are not interested in the details.

### Derivation of the Vectorized Objective Function Derivative

The objective derivative we want to derive is:

$$\frac{\partial}{\partial w} L(w) = \frac{2}{n} \Phi^T(\Phi w - y)$$

Substitute $\Phi = \begin{bmatrix} \phi(x_1)^T \\ \phi(x_2)^T \\ \vdots \\ \phi(x_n)^T \end{bmatrix}$ and $y = \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}$ into the objective derivative:

$$\frac{\partial}{\partial w} \frac{1}{n} (\Phi w - y)^2 = \frac{2}{n} \begin{bmatrix} \phi(x_1)^T \\ \phi(x_2)^T \\ \vdots \\ \phi(x_n)^T \end{bmatrix}^T \left( \begin{bmatrix} \phi(x_1)^T \\ \phi(x_2)^T \\ \vdots \\ \phi(x_n)^T \end{bmatrix} w - \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix} \right)$$

Expand the matrix multiplication:

$$\frac{\partial}{\partial w} \frac{1}{n} (\Phi w - y)^2 = \frac{2}{n} \begin{bmatrix} \phi(x_1) & \phi(x_2) & \cdots & \phi(x_n) \end{bmatrix} \left( \begin{bmatrix} \phi(x_1)^T \\ \phi(x_2)^T \\ \vdots \\ \phi(x_n)^T \end{bmatrix} w - \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix} \right)$$

combine $\Phi w - y$:

$$\frac{\partial}{\partial w} \frac{1}{n} (\Phi w - y)^2 = \frac{2}{n} \begin{bmatrix} \phi(x_1) & \phi(x_2) & \cdots & \phi(x_n) \end{bmatrix} \left( \begin{bmatrix} \phi(x_1)^T w - y_1 \\ \phi(x_2)^T w - y_2 \\ \vdots \\ \phi(x_n)^T w - y_n \end{bmatrix} \right)$$

Distribute the matrix multiplication:

$$\frac{\partial}{\partial w} L(w) = \frac{2}{n} \left[ \phi(x_1) (\phi(x_1)^T w - y_1) + \phi(x_2) (\phi(x_2)^T w - y_2) + \cdots + \phi(x_n) (\phi(x_n)^T w - y_n) \right] $$

Simply the equation:

$$ \frac{\partial}{\partial w} L(w)= \frac{2}{n} \sum_{i=1}^{n} \phi(x_i) (\phi(x_i)^T w - y_i)$$

Thus:
$$
\frac{2}{n} \Phi^T(\Phi w - y) \equiv \frac{2}{n}  \sum_{i=1}^{n} \phi(x_i) (\phi(x_i)^T w - y_i)
$$


### Okay that was a lot of math, let's get to the code!

Lets implement $\frac{\partial}{\partial w} L(w) = \frac{2}{n} \Phi^T(\Phi w - y)$ and use it to build our gradient descent algorithm.

In [5]:
def ߜL(w: ndarray, Φ: ndarray, y: ndarray) -> ndarray:
    """
    Calculates the gradient of the loss function L with respect to weights (w).

    Parameters:
    - w: ndarray, The weight vector.
    - Φ: ndarray, The feature matrix after applying the basis function. (Phi)
    - y: ndarray, The target values.

    Returns:
    - ndarray: Gradient of the loss function L with respect to w.
    """
    n, m = Φ.shape
    return 1/n * Φ.T.dot(Φ.dot(w) - y)

def gradient_descent(Φ: ndarray, y: ndarray, α: float = 0.01, num_iter: int = 1000) -> ndarray:
    """
    Performs gradient descent to optimize w.

    Parameters:
    - Φ: ndarray, The feature matrix (Phi).
    - y: ndarray, The target values.
    - α: float, The learning rate.
    - num_iter: int, The number of training iterations.

    Returns:
    - ndarray: The optimized weights vector.
    """
    n, m = Φ.shape
    w = np.zeros((m, 1))
    for _ in range(num_iter):
        gradient = ߜL(w, Φ, y)  # Gradient of L with respect to w
        
        # Checking for convergence (See note below)
        if np.all(np.abs(gradient) < 1e-5) or np.isnan(gradient).any():
            break
            
        # Check for gradient explosion (See note below)
        if np.isinf(gradient).any(): 
            raise ValueError("Gradient exploded")

        w -= α * gradient
    return w

def predict(Φ: ndarray, w: ndarray) -> ndarray:
    """
    Predicts the target values using the linear model.

    Parameters:
    - Φ: ndarray, The feature matrix. (Phi)
    - w: ndarray, The weights vector.

    Returns:
    - ndarray: Predicted values.
    """
    return Φ.dot(w)

### Comments on Gradient Descent Implementation

I added a few lines that may look foreign to some of you. Specifically, the "checking for convergence" and "gradient explosion" sections. These are important to ensure that our gradient descent algorithm is working properly. 

Some of you may have experienced your weight vector filling with values equal to `np.inf`. This is a result of your gradients walking you further and further away from the minimum at each step. If this happens, you will need to adjust your learning rate. 

Conversly, if your weight vector seems to include `np.nan` values, this is a result of your gradients being too small. This either may mean you are at the minimum and you can stop training, or your learning rate is too small.

### Defining a Closed Form Solution For Linear Regression

As we learned in class, since the linear regression objective function is convex, we can solve for the optimal weight vector using the closed form solution:

$$w = (\Phi^T\Phi)^{-1}\Phi^Ty$$

Like above, $\Phi$ denotes the feature matrix and \(y\) is the vector of target values.

In [6]:
def closed_form_solution(Φ: ndarray, y: ndarray) -> ndarray:
    """
    Computes the closed form solution for linear regression weights.

    Parameters:
    - Φ: ndarray, The feature matrix after applying the basis function. (Phi)
    - y: ndarray, The target values.

    Returns:
    - ndarray: The weight vector that minimizes the loss function.
    """
    return np.linalg.inv(Φ.T.dot(Φ)).dot(Φ.T).dot(y)

def mse(y: ndarray, y_hat: ndarray) -> float:
    """
    Calculates the Mean Squared Error (MSE) between actual and predicted values.

    Parameters:
    - y: ndarray, Actual target values.
    - y_hat: ndarray, Predicted values by the model.

    Returns:
    - float: The mean squared error between actual and predicted values.
    """
    return np.mean((y - y_hat)**2)

### Training a Linear Regression Model

Before we can train or predict with our model, we need to implement to make sure our data is in the correct format. Specifically, we need to feature map our data to add a bias term!

In [7]:
def linear_feature_map(X):
    return np.hstack((np.ones((X.shape[0], 1)), X))

In [8]:
X_linear_train = linear_feature_map(X_train)
X_linear_val = linear_feature_map(X_val)

### Solving with Gradient Descent

In [9]:
linear_gd_w = gradient_descent(X_linear_train, y_train, num_iter=10_000, α=0.01)
pred_train = predict(X_linear_train, linear_gd_w)
print(f"My Linear GD Train MSE: {mse(y_train, pred_train)}")
pred_val = predict(X_linear_val, linear_gd_w)
print(f"My Linear GD Validation MSE: {mse(y_val, pred_val)}")

My Linear GD Train MSE: 0.041752878953118994
My Linear GD Validation MSE: 0.05992399950038505


### Solving with Closed Form Solution

In [10]:
linear_cfs_w = closed_form_solution(X_linear_train, y_train)
pred_train = predict(X_linear_train, linear_cfs_w)
print(f"My Linear Closed Form Train MSE: {mse(y_train, pred_train)}")
pred_val = predict(X_linear_val, linear_cfs_w)
print(f"My Linear Closed Form Validation MSE: {mse(y_val, pred_val)}")

My Linear Closed Form Train MSE: 0.041752878453990054
My Linear Closed Form Validation MSE: 0.05992272985435261


### Using Sklearn Linear Regression

No need to linear feature map X as SKlearn does it for us.

In [11]:
sk_lr = SKLinearRegression() 
sk_lr.fit(X_train,y_train.flatten()) # y is 2D, but scikit-learn expects 1D
pred_train = sk_lr.predict(X_train).reshape(-1,1)
print(f"SKLearn Linear Train MSE: {mse(y_train, pred_train)}")
pred_val = sk_lr.predict(X_val).reshape(-1,1)
print(f"SKLearn Linear Validation MSE: {mse(y_val, pred_val)}")

SKLearn Linear Train MSE: 0.04175287845399005
SKLearn Linear Validation MSE: 0.05992272985435297


### Polynomial Regression time!

Instead of implementing a polynomial feature map, like we did for our linear model, we can use SKLearn's `PolynomialFeatures` to do it for us. This will save us a lot of time and effort.

In [12]:
X_poly_train = PolynomialFeatures(degree=2, include_bias=True).fit_transform(X_train)
X_poly_val = PolynomialFeatures(degree=2, include_bias=True).fit_transform(X_val)

### Training a Polynomial Regression Model (Gradient Descent)

Now that we have feature mapped our data to include polynomial features, we can train our polynomial regression model using gradient descent. Because of the feature mapping, we don't need to make any changes to our gradient descent algorithm, we can use it as is. This is the beauty of feature mapping!

In [13]:
poly_gd_w = gradient_descent(X_poly_train, y_train, num_iter=10_000, α=0.01)
pred_train = predict(X_poly_train, poly_gd_w)
print(f"My Polynomial GD Train MSE: {mse(y_train, pred_train)}")
pred_val = predict(X_poly_val, poly_gd_w)
print(f"My Polynomial GD Validation MSE: {mse(y_val, pred_val)}")

My Polynomial GD Train MSE: 0.03166616796874941
My Polynomial GD Validation MSE: 0.09333443610453591


### Training a Polynomial Regression Model (Closed Form Solution)

In [14]:
poly_cfs_w = closed_form_solution(X_poly_train, y_train)
pred_train = predict(X_poly_train, poly_cfs_w)
print(f"My Polynomial Closed Form Train MSE: {mse(y_train, pred_train)}")
pred_val = predict(X_poly_val, poly_cfs_w)
print(f"My Polynomial Closed Form Validation MSE: {mse(y_val, pred_val)}")

My Polynomial Closed Form Train MSE: 0.03166484523384303
My Polynomial Closed Form Validation MSE: 0.09294325090399753


In [15]:
sk_poly_lr = SKLinearRegression()
sk_poly_lr.fit(X_poly_train,y_train.flatten()) # y is 2D, but scikit-learn expects 1D
pred_train = sk_poly_lr.predict(X_poly_train).reshape(-1,1) # SKLinearRegression.predict() returns 1D array, but we want 2D for my mse function
print(f"SKLearn Polynomial Train MSE: {mse(y_train, pred_train)}")
pred_val = sk_poly_lr.predict(X_poly_val).reshape(-1,1)
print(f"SKLearn Polynomial Validation MSE: {mse(y_val, pred_val)}")

SKLearn Polynomial Train MSE: 0.031664845233843046
SKLearn Polynomial Validation MSE: 0.09294325090399327


### Final comparison of models

In [16]:
print(f'Linear Closed Form MSE on Validation Set: {mse(y_val, predict(X_linear_val, linear_cfs_w))}')
print(f'Polynomial Closed Form MSE on Validation Set: {mse(y_val, predict(X_poly_val, poly_cfs_w))}')

Linear Closed Form MSE on Validation Set: 0.05992272985435261
Polynomial Closed Form MSE on Validation Set: 0.09294325090399753


### What model should we use? Polynomial or Linear?

While the polynomial seems to fit better to the training data, it doesn't seem to generalize well to the validation data (low train mse, high validation mse). The linear model seems to generalize better to the validation data (higher train mse, low validation mse). Moreover, the linear model seems to be better here.

We can end our experiment here by testing the better model on the test data. This score will give us an idea of how well our model will perform on future data and since we have not used the test data to train or select our model, we can be confident in the score.

In [17]:
print(f'Linear Closed Form MSE on Test Set: {mse(y_test, predict(linear_feature_map(X_test), linear_cfs_w))}')

Linear Closed Form MSE on Test Set: 0.04931245936959388
